# Transit Accessibility Bar Chart

Shows transit stops per 1,000 residents across Toronto neighbourhoods.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_amenities` | neighbourhood × year | transit_per_1000, transit_index, transit_count |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    transit_per_1000,
    transit_index,
    transit_count,
    population,
    amenity_tier
FROM mart_toronto.mart_neighbourhood_amenities
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_amenities)
  AND transit_per_1000 IS NOT NULL
ORDER BY transit_per_1000 DESC
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} neighbourhoods")

Loaded 158 neighbourhoods


### Transformation Steps

1. Sort by transit accessibility
2. Select top 20 for visualization

In [2]:
data = df.head(20).to_dict("records")

### Sample Output

In [3]:
df[["neighbourhood_name", "transit_per_1000", "transit_index", "transit_count"]].head(
    10
)

,neighbourhood_name,transit_per_1000,transit_index,transit_count
0,West Humber-Clairville,9.550,288.0,318
1,Humber Summit,9.110,274.8,111
2,Bridle Path-Sunnybrook-York Mills,8.166,246.3,73
3,Yorkdale-Glen Park,7.880,237.7,131
4,Beechborough-Greenbrook,7.668,231.3,48
5,York University Heights,7.645,230.6,216
6,St.Andrew-Windfields,7.017,211.6,119
7,Old East York,6.332,191.0,58
8,Milliken,6.172,186.1,148
9,Etobicoke City Centre,6.102,184.0,144


## 2. Data Visualization

### Figure Factory

Uses `create_horizontal_bar` from `portfolio_app.figures.toronto.bar_charts`.

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.bar_charts import create_horizontal_bar

fig = create_horizontal_bar(
    data=data,
    name_column="neighbourhood_name",
    value_column="transit_per_1000",
    title="Top 20 Neighbourhoods by Transit Accessibility",
    color="#00BCD4",
    value_format=".2f",
)

fig.show()

### Transit Statistics

In [5]:
print("City-wide Transit Statistics:")
print(f"  Total Transit Stops: {df['transit_count'].sum():,.0f}")
print(f"  Average per 1,000 pop: {df['transit_per_1000'].mean():.2f}")
print(f"  Median per 1,000 pop: {df['transit_per_1000'].median():.2f}")
print(f"  Best Access: {df['transit_per_1000'].max():.2f} per 1,000")
print(f"  Worst Access: {df['transit_per_1000'].min():.2f} per 1,000")

City-wide Transit Statistics:
  Total Transit Stops: 9,065
  Average per 1,000 pop: 3.32
  Median per 1,000 pop: 3.09
  Best Access: 9.55 per 1,000
  Worst Access: 0.53 per 1,000
